# 💳 SmartSentry AML — Notebook 2a: Merging & Typologies
---
Generates legitimate and fraud transactions across **11 typologies** (8 original + 3 new inactive-user reactivation patterns), merges and rebalances to the target fraud rate, then enriches with entity FK-joins, IP-graph features, and shared-identity markers.

## ✨ Inactive-User Feature — Design Summary

| Item | Detail |
|---|---|
| **Pool size** | 5% of all accounts (`INACTIVE_ACCOUNT_FRAC = 0.05`) |
| **No-overlap guarantee** | `inactive_account_set ∩ dormant_account_set = ∅` enforced at sampling |
| **Definition of inactive** | No transactions for ≥ 45 days (`INACTIVE_GAP_DAYS`) before reactivation |
| **Account lifecycle** | Early cluster (days 0–EARLY_END) → silence (≥45 days) → reactivation burst |
| **`no_of_inactive_days`** | New integer column on **every** row. `NaN` for non-inactive accounts; `0` for pre-silence early rows; actual gap days (≥45) for reactivation rows |
| **Fraud sub-patterns** | `inactive_ATO` (35%), `inactive_smurfing` (20%), `inactive_to_offshore` (20%) |
| **Clean reactivation** | 25% of inactive accounts reactivate legitimately — `label=0` but still carry `no_of_inactive_days` |
| **IP signal** | `inactive_ATO` rows always assigned `ip_type='risk'` (adversary control) |
| **Shared identity** | Each inactive sub-pattern gets distinct KYC / phone / email group IDs |

**Inputs :** `outputs/customers.csv`, `accounts.csv`, `devices.csv`, `beneficiaries.csv`

**Outputs :**
- `outputs/txns_stage1.parquet` — full enriched transaction table  
- `outputs/dormant_account_set.csv` — dormant account IDs (for Notebook 2b)  
- `outputs/inactive_account_set.csv` — inactive account IDs + last active timestamp


In [ ]:
# ── A-1: Imports ─────────────────────────────────────────────────────────────
import random
import warnings
import hashlib
from datetime import datetime, timedelta
from pathlib import Path

import numpy as np
import pandas as pd

warnings.filterwarnings('ignore')

OUTPUT_DIR = Path('./outputs')
OUTPUT_DIR.mkdir(exist_ok=True)

print('✅ Libraries loaded')

In [ ]:
# ── A-2: Configuration ────────────────────────────────────────────────────────
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

START_DATE      = datetime(2025, 9, 1)
END_DATE        = datetime(2025, 12, 31)
DATE_RANGE_DAYS = (END_DATE - START_DATE).days

TARGET_FRAUD_RATE            = 0.04
AVG_TXNS_PER_ACCOUNT_PER_DAY = 50 / DATE_RANGE_DAYS
AVG_TXNS_PER_ACCOUNT         = AVG_TXNS_PER_ACCOUNT_PER_DAY * DATE_RANGE_DAYS

# ── Dormant account config (UNCHANGED) ───────────────────────────────────────
DORMANT_ACCOUNT_FRAC  = 0.05
_dorm_split_end       = max(25, int(DATE_RANGE_DAYS * 0.20))
_dorm_second_start    = max(_dorm_split_end + 35, int(DATE_RANGE_DAYS * 0.55))
DORMANT_FIRST_WINDOW  = (0, _dorm_split_end)
DORMANT_SECOND_WINDOW = (_dorm_second_start, DATE_RANGE_DAYS)
DORMANT_FIRST_TXNS    = (3, 8)
DORMANT_SECOND_TXNS   = (5, 15)

# ── Inactive account config (NEW) ─────────────────────────────────────────────
#
# Lifecycle:
#   Phase 1 (active)  : days 0 .. INACTIVE_EARLY_END    → small legit cluster
#   Phase 2 (silent)  : INACTIVE_EARLY_END .. INACTIVE_REACT_START (≥ 45 days)
#   Phase 3 (reactiv.): INACTIVE_REACT_START .. DATE_RANGE_DAYS  → risky or clean
#
INACTIVE_ACCOUNT_FRAC  = 0.05          # 5 % of all accounts
INACTIVE_GAP_DAYS      = 45            # minimum silence before reactivation
INACTIVE_EARLY_END     = max(20, int(DATE_RANGE_DAYS * 0.15))        # e.g. day 18
INACTIVE_REACT_START   = INACTIVE_EARLY_END + INACTIVE_GAP_DAYS      # e.g. day 63
INACTIVE_EARLY_TXNS    = (2, 6)        # transactions before going silent

# Sub-pattern allocation — MUST sum to exactly 1.0
# These are mutually exclusive slices of inactive_account_set
INACTIVE_FRAUD_FRACS = {
    'inactive_ATO'         : 0.35,   # foreign device + large off-hours transfer
    'inactive_smurfing'    : 0.20,   # cash structuring burst just below threshold
    'inactive_to_offshore' : 0.20,   # wire to crypto / offshore after silence
    'inactive_legit'       : 0.25,   # clean reactivation (label=0)
}
assert abs(sum(INACTIVE_FRAUD_FRACS.values()) - 1.0) < 1e-9, 'Fracs must sum to 1.0'

# ── Channel / TXN config (UNCHANGED) ─────────────────────────────────────────
CHANNELS      = ['mobile', 'web', 'branch', 'atm']
CHANNEL_PROBS = [0.50, 0.30, 0.10, 0.10]
TXN_TYPES_BY_CHANNEL = {
    'mobile': ['UPI', 'IMPS', 'wallet_transfer'],
    'web':    ['NEFT', 'RTGS', 'online_transfer'],
    'branch': ['cash_deposit', 'cash_withdrawal', 'DD'],
    'atm':    ['cash_withdrawal', 'balance_enquiry'],
}
CASH_TXN_TYPES = {'cash_deposit', 'cash_withdrawal', 'DD'}

AMOUNT_BY_KYC = {
    'low':    {'mean': 7.0, 'sigma': 1.0},
    'medium': {'mean': 8.0, 'sigma': 1.0},
    'high':   {'mean': 9.5, 'sigma': 1.0},
}
AMOUNT_DEFAULT         = {'mean': 8.0, 'sigma': 1.0}
INTERNAL_TRANSFER_PROB = 0.40

HOUR_WEIGHTS_NORMAL = [1,1,1,1,1,1,2,3,5,6,6,6,5,6,6,6,5,5,4,4,3,2,2,1]
HOUR_WEIGHTS_FRAUD  = [4,4,4,4,3,2,1,1,1,1,1,1,1,1,1,1,2,2,3,3,4,4,4,4]

_scale = DATE_RANGE_DAYS / 121
FRAUD_VOLUMES = {
    # ── Original 8 active / dormant typologies (UNCHANGED) ───────────────
    'mule_ring'           : {'num_rings'          : max(1,int(60*_scale)),
                             'accounts_per_ring'  : 10,
                             'txns_per_account'   : 8,
                             'day_min'            : int(DATE_RANGE_DAYS*0.50),
                             'day_max'            : int(DATE_RANGE_DAYS*0.99),
                             'amount_mean'        : 9.5,
                             'amount_sigma'       : 0.5,
                             'channels'           : ['mobile','web']},
    'layering'            : {'num_chains'         : max(1,int(900*_scale)),
                             'chain_length'       : 3,
                             'day_min'            : int(DATE_RANGE_DAYS*0.60),
                             'day_max'            : int(DATE_RANGE_DAYS*0.99),
                             'hop_min_mins'       : 5,
                             'hop_max_mins'       : 30,
                             'amount_mean_base'   : 9.0,
                             'amount_mean_decay'  : 0.2,
                             'amount_sigma'       : 0.8,
                             'channels'           : ['mobile','web']},
    'ATO'                 : {'count'              : max(1,int(2000*_scale)),
                             'day_min'            : int(DATE_RANGE_DAYS*0.70),
                             'day_max'            : int(DATE_RANGE_DAYS*0.99),
                             'amount_mean'        : 10.5,
                             'amount_sigma'       : 0.8,
                             'channels'           : ['mobile','web']},
    'smurfing'            : {'num_groups'         : max(1,int(400*_scale)),
                             'min_txns'           : 4,
                             'max_txns'           : 8,
                             'amount_min'         : 8500,
                             'amount_max'         : 9999,
                             'day_window'         : 2,
                             'channels'           : ['branch','atm'],
                             'force_cash_flag'    : True},
    'identity_fraud'      : {'count'              : max(1,int(800*_scale)),
                             'max_account_days'   : 60,
                             'day_min'            : 0,
                             'day_max'            : int(DATE_RANGE_DAYS*0.25),
                             'amount_mean'        : 11.0,
                             'amount_sigma'       : 0.6,
                             'channels'           : ['mobile','web']},
    'dormant_ATO'         : {'frac_dormant_targeted': 0.60,
                             'txns_per_account'   : (2,5),
                             'amount_mean'        : 11.0,
                             'amount_sigma'       : 0.7,
                             'channels'           : ['mobile','web']},
    'dormant_smurfing'    : {'frac_dormant_targeted': 0.25,
                             'min_txns'           : 3,
                             'max_txns'           : 6,
                             'amount_min'         : 8500,
                             'amount_max'         : 9999,
                             'channels'           : ['branch','atm'],
                             'force_cash_flag'    : True,
                             'day_window'         : 3},
    'dormant_to_offshore' : {'frac_dormant_targeted': 0.30,
                             'txns_per_account'   : (1,3),
                             'amount_mean'        : 11.5,
                             'amount_sigma'       : 0.5,
                             'channels'           : ['web','mobile']},
    # ── NEW: 3 inactive-user fraud typologies ─────────────────────────────
    # Applied ONLY to inactive_account_set; zero overlap with dormant / active.
    'inactive_ATO'        : {'txns_per_account'   : (3, 7),
                             'amount_mean'        : 11.2,
                             'amount_sigma'       : 0.9,
                             'channels'           : ['mobile','web'],
                             'new_device'         : True},   # foreign device flag
    'inactive_smurfing'   : {'min_txns'           : 3,
                             'max_txns'           : 7,
                             'amount_min'         : 8000,
                             'amount_max'         : 9999,
                             'channels'           : ['branch','atm'],
                             'force_cash_flag'    : True,
                             'day_window'         : 3},
    'inactive_to_offshore': {'txns_per_account'   : (1, 4),
                             'amount_mean'        : 12.0,
                             'amount_sigma'       : 0.5,
                             'channels'           : ['web','mobile']},
}

IP_HOME_FRAC          = 0.70
IP_ROAM_FRAC          = 0.20
IP_HOME_BASE_SCORE    = 0.05
IP_ROAM_BASE_SCORE    = 0.30
IP_RISK_BASE_SCORE    = 0.80
HIGH_RISK_IP_PREFIXES = ['185.220','45.142','198.96','176.10','162.247',
                         '91.108','77.247','104.244','194.165','185.107']

print('✅ Configuration loaded')
print(f'   Date range           : {START_DATE.date()} → {END_DATE.date()} ({DATE_RANGE_DAYS} days)')
print(f'   Fraud target         : {TARGET_FRAUD_RATE:.0%}')
print(f'   Dormant accounts     : {DORMANT_ACCOUNT_FRAC:.0%}')
print(f'   Inactive accounts    : {INACTIVE_ACCOUNT_FRAC:.0%}  (gap ≥ {INACTIVE_GAP_DAYS} days)')
print(f'   Inactive early end   : day {INACTIVE_EARLY_END}')
print(f'   Inactive react start : day {INACTIVE_REACT_START}')
print(f'   Dormant window 1     : days {DORMANT_FIRST_WINDOW[0]}–{DORMANT_FIRST_WINDOW[1]}')
print(f'   Dormant window 2     : days {DORMANT_SECOND_WINDOW[0]}–{DORMANT_SECOND_WINDOW[1]}')

In [ ]:
# ── A-3: Load Reference Tables ───────────────────────────────────────────────
customers     = pd.read_csv(OUTPUT_DIR / 'customers.csv')
accounts      = pd.read_csv(OUTPUT_DIR / 'accounts.csv')
devices       = pd.read_csv(OUTPUT_DIR / 'devices.csv')
beneficiaries = pd.read_csv(OUTPUT_DIR / 'beneficiaries.csv')

device_list        = devices['device_id'].tolist()
account_device_map = {acc: random.choice(device_list) for acc in accounts['account_id']}

bene_list = beneficiaries['beneficiary_id'].tolist()
account_beneficiaries_map = {
    acc: random.sample(bene_list, k=random.randint(2, 5))
    for acc in accounts['account_id']
}

high_risk_mask = (
    beneficiaries['beneficiary_type'].isin({'crypto', 'offshore'}) |
    (beneficiaries['beneficiary_country_risk'] == 'high')
)
high_risk_bene_pool  = beneficiaries.loc[high_risk_mask, 'beneficiary_id'].tolist()
offshore_crypto_pool = beneficiaries.loc[
    beneficiaries['beneficiary_type'].isin({'crypto', 'offshore'}),
    'beneficiary_id'
].tolist()

all_account_ids = accounts['account_id'].tolist()

# ── Dormant account set (original logic) ─────────────────────────────────────
n_dormant           = max(1, int(len(all_account_ids) * DORMANT_ACCOUNT_FRAC))
dormant_account_set = set(random.sample(all_account_ids, n_dormant))

# ── Inactive account set — sampled from NON-dormant accounts only ─────────────
eligible_for_inactive = [a for a in all_account_ids if a not in dormant_account_set]
n_inactive            = max(1, int(len(all_account_ids) * INACTIVE_ACCOUNT_FRAC))
inactive_account_set  = set(random.sample(eligible_for_inactive, n_inactive))

# Hard guarantee: zero overlap
assert dormant_account_set.isdisjoint(inactive_account_set), \
    '❌ dormant ∩ inactive overlap detected'

if 'home_lat' in accounts.columns:
    cust_geo_map = (
        accounts[['customer_id','home_lat','home_lon']]
        .drop_duplicates('customer_id')
        .set_index('customer_id')
    )
else:
    cust_geo_map = None

acc_open_date_map = {
    row['account_id']: START_DATE.date() - timedelta(days=int(row['account_open_days']))
    for _, row in accounts.iterrows()
}

print('✅ Reference tables loaded')
print(f'   customers: {len(customers):,} | accounts: {len(accounts):,} | '
      f'devices: {len(devices):,} | beneficiaries: {len(beneficiaries):,}')
print(f'   high_risk_bene_pool  : {len(high_risk_bene_pool):,}')
print(f'   dormant_account_set  : {len(dormant_account_set):,} '
      f'({len(dormant_account_set)/len(all_account_ids):.1%} of accounts)')
print(f'   inactive_account_set : {len(inactive_account_set):,} '
      f'({len(inactive_account_set)/len(all_account_ids):.1%} of accounts)')
print(f'   Overlap check        : ✅ dormant ∩ inactive = 0')

In [ ]:
# ── A-4: Shared Transaction Builder ──────────────────────────────────────────
def weighted_hour(fraud=False):
    weights = HOUR_WEIGHTS_FRAUD if fraud else HOUR_WEIGHTS_NORMAL
    return random.choices(range(24), weights=weights)[0]

def random_ts(fraud=False, day_min=0, day_max=None):
    """Return a random datetime within the simulation window."""
    if day_max is None:
        day_max = DATE_RANGE_DAYS
    day_max = min(day_max, DATE_RANGE_DAYS)
    return START_DATE + timedelta(
        days    = random.randint(day_min, day_max),
        hours   = weighted_hour(fraud),
        minutes = random.randint(0, 59),
        seconds = random.randint(0, 59),
    )

def get_txn_type(channel):
    return random.choice(TXN_TYPES_BY_CHANNEL.get(channel, ['online_transfer']))

def is_cash(txn_type):
    return int(txn_type in CASH_TXN_TYPES)

TXN_COLS = [
    'transaction_id', 'customer_id', 'sender_account_id', 'receiver_account_id',
    'beneficiary_id', 'device_id', 'timestamp', 'amount', 'channel', 'debit_credit',
    'transaction_type', 'cash_flag', 'synthetic_flow_id', 'flow_depth', 'hop_number',
    'time_since_origin_ts', 'fraud_type', 'label',
]
_FRAUD_TYPE_IDX    = TXN_COLS.index('fraud_type')
_CASH_FLAG_IDX     = TXN_COLS.index('cash_flag')
_SENDER_ACCT_IDX   = TXN_COLS.index('sender_account_id')

def build_row(txn_id, customer_id, sender_account_id, device_id,
              ts, amount, channel, fraud_type, label,
              receiver_account_id=None, beneficiary_id=None,
              synthetic_flow_id=None, flow_depth=None,
              hop_number=None, time_since_origin_ts=None,
              force_cash=False):
    txn_type = get_txn_type(channel)
    cash     = 1 if force_cash else is_cash(txn_type)
    return [
        f'T{txn_id}', customer_id, sender_account_id, receiver_account_id,
        beneficiary_id, device_id, ts, round(amount, 2), channel, 'debit',
        txn_type, cash, synthetic_flow_id, flow_depth, hop_number,
        time_since_origin_ts, fraud_type, label,
    ]

print(f'✅ Transaction builder ready  ({len(TXN_COLS)} columns)')

In [ ]:
# ── A-5: Generate Legitimate Transactions ────────────────────────────────────
#
# Three account populations:
#   (a) Normal active   → full Poisson-distributed history
#   (b) Dormant         → early cluster + late cluster  (original logic)
#   (c) Inactive        → EARLY CLUSTER ONLY here (pre-silence phase)
#                         Reactivation rows injected in Cell A-7.

acc_meta    = accounts.set_index('account_id')
legit_rows  = []
txn_id      = 0

def _legit_row(acc, cust, device, kyc, day_min, day_max, txn_id):
    params = AMOUNT_BY_KYC.get(str(kyc).lower(), AMOUNT_DEFAULT)
    ts     = random_ts(fraud=False, day_min=day_min, day_max=day_max)
    amount = np.random.lognormal(mean=params['mean'], sigma=params['sigma'])
    ch     = random.choices(CHANNELS, weights=CHANNEL_PROBS)[0]
    if random.random() < INTERNAL_TRANSFER_PROB:
        return build_row(txn_id, cust, acc, device, ts, amount, ch, 'normal', 0,
                         receiver_account_id=random.choice(all_account_ids))
    else:
        return build_row(txn_id, cust, acc, device, ts, amount, ch, 'normal', 0,
                         beneficiary_id=random.choice(account_beneficiaries_map[acc]))

dormant_count_total  = 0
inactive_early_count = 0

for acc in accounts['account_id']:
    kyc    = acc_meta.loc[acc, 'kyc_level']
    cust   = acc_meta.loc[acc, 'customer_id']
    device = account_device_map[acc]

    if acc in dormant_account_set:
        # ── Dormant: two clusters separated by long silence (original) ────
        n_first  = random.randint(*DORMANT_FIRST_TXNS)
        n_second = random.randint(*DORMANT_SECOND_TXNS)
        for _ in range(n_first):
            legit_rows.append(_legit_row(acc, cust, device, kyc,
                                         DORMANT_FIRST_WINDOW[0],
                                         DORMANT_FIRST_WINDOW[1], txn_id))
            txn_id += 1
        for _ in range(n_second):
            legit_rows.append(_legit_row(acc, cust, device, kyc,
                                         DORMANT_SECOND_WINDOW[0],
                                         DORMANT_SECOND_WINDOW[1], txn_id))
            txn_id += 1
        dormant_count_total += n_first + n_second

    elif acc in inactive_account_set:
        # ── Inactive: ONLY pre-silence early cluster here ─────────────────
        n_early = random.randint(*INACTIVE_EARLY_TXNS)
        for _ in range(n_early):
            legit_rows.append(_legit_row(acc, cust, device, kyc,
                                         0, INACTIVE_EARLY_END, txn_id))
            txn_id += 1
        inactive_early_count += n_early

    else:
        # ── Normal active account ──────────────────────────────────────────
        n_txns = np.random.poisson(AVG_TXNS_PER_ACCOUNT)
        for _ in range(n_txns):
            legit_rows.append(_legit_row(acc, cust, device, kyc,
                                         0, DATE_RANGE_DAYS, txn_id))
            txn_id += 1

legit_df  = pd.DataFrame(legit_rows, columns=TXN_COLS)
n_active  = len(all_account_ids) - len(dormant_account_set) - len(inactive_account_set)

print(f'✅ Legitimate transactions : {len(legit_df):,} rows')
print(f'   Active accounts          : {n_active:,}')
print(f'   Dormant accounts         : {len(dormant_account_set):,}  ({dormant_count_total:,} rows)')
print(f'   Inactive accts (early Φ1): {len(inactive_account_set):,}  ({inactive_early_count:,} rows — pre-silence only)')
print(f'   Internal transfers       : {legit_df["receiver_account_id"].notna().sum():,}')
print(f'   External transfers       : {legit_df["beneficiary_id"].notna().sum():,}')
legit_df.head(3)

In [ ]:
# ── A-6: Inject Fraud Typologies (Original 8 — Active & Dormant only) ────────
#
# Sampling pool explicitly EXCLUDES inactive_account_set.
# This guarantees the 8 original typologies never touch inactive accounts.

acc_meta_fraud  = accounts.set_index('account_id')
active_pool     = [a for a in accounts['account_id'].tolist()
                   if a not in inactive_account_set]       # dormant still included
all_devs        = devices['device_id'].tolist()
fraud_rows      = []

# ── 6A: MULE RINGS ────────────────────────────────────────────────────────────
cfg = FRAUD_VOLUMES['mule_ring']
_mule_flow_counter = 0
for _ in range(cfg['num_rings']):
    mule_accs      = random.sample(active_pool, cfg['accounts_per_ring'])
    shared_dev     = random.choice(all_devs)
    exit_bene      = random.choice(high_risk_bene_pool)
    flow_id        = f'FLOW_{_mule_flow_counter:05d}'
    ring_depth     = len(mule_accs)
    ring_origin_ts = random_ts(True, cfg['day_min'], cfg['day_max'])
    _mule_flow_counter += 1
    for hop_i, acc in enumerate(mule_accs, start=1):
        cust = acc_meta_fraud.loc[acc, 'customer_id']
        for _ in range(cfg['txns_per_account']):
            ts  = random_ts(True, cfg['day_min'], cfg['day_max'])
            amt = np.random.lognormal(cfg['amount_mean'], cfg['amount_sigma'])
            ch  = random.choice(cfg['channels'])
            fraud_rows.append(build_row(
                txn_id, cust, acc, shared_dev, ts, amt, ch, 'mule_ring', 1,
                beneficiary_id=exit_bene,
                synthetic_flow_id=flow_id, flow_depth=ring_depth,
                hop_number=hop_i, time_since_origin_ts=ring_origin_ts,
            ))
            txn_id += 1

_mr_count = sum(1 for r in fraud_rows if r[_FRAUD_TYPE_IDX] == 'mule_ring')
print(f'  Mule rings          : {_mr_count:>6,} rows')

# ── 6B: LAYERING CHAINS ───────────────────────────────────────────────────────
cfg = FRAUD_VOLUMES['layering']
_layer_flow_counter = 10_000
for _ in range(cfg['num_chains']):
    chain       = random.sample(active_pool, cfg['chain_length']) + [None]
    chain[-1]   = chain[0]
    base_ts     = random_ts(True, cfg['day_min'], cfg['day_max'])
    flow_id     = f'FLOW_{_layer_flow_counter:05d}'
    chain_depth = cfg['chain_length']
    _layer_flow_counter += 1
    for i in range(cfg['chain_length']):
        sender   = chain[i]
        receiver = chain[i + 1]
        cust     = acc_meta_fraud.loc[sender, 'customer_id']
        device   = account_device_map[sender]
        ts       = base_ts + timedelta(
                       minutes=random.randint(cfg['hop_min_mins'],
                                             cfg['hop_max_mins']) * (i + 1))
        amt = np.random.lognormal(
            cfg['amount_mean_base'] - i * cfg['amount_mean_decay'],
            cfg['amount_sigma'])
        ch = random.choice(cfg['channels'])
        fraud_rows.append(build_row(
            txn_id, cust, sender, device, ts, amt, ch, 'layering', 1,
            receiver_account_id=receiver,
            synthetic_flow_id=flow_id, flow_depth=chain_depth,
            hop_number=i + 1, time_since_origin_ts=base_ts,
        ))
        txn_id += 1

_ly_count = sum(1 for r in fraud_rows if r[_FRAUD_TYPE_IDX] == 'layering')
print(f'  Layering chains     : {_ly_count:>6,} rows')

# ── 6C: ACCOUNT TAKEOVER (ATO) ────────────────────────────────────────────────
cfg = FRAUD_VOLUMES['ATO']
for _ in range(cfg['count']):
    acc       = random.choice(active_pool)
    cust      = acc_meta_fraud.loc[acc, 'customer_id']
    new_dev   = random.choice(all_devs)
    exit_bene = random.choice(high_risk_bene_pool)
    ts        = random_ts(True, cfg['day_min'], cfg['day_max'])
    amt       = np.random.lognormal(cfg['amount_mean'], cfg['amount_sigma'])
    ch        = random.choice(cfg['channels'])
    fraud_rows.append(build_row(txn_id, cust, acc, new_dev, ts, amt, ch, 'ATO', 1,
                                beneficiary_id=exit_bene))
    txn_id += 1

_ato_count = sum(1 for r in fraud_rows if r[_FRAUD_TYPE_IDX] == 'ATO')
print(f'  ATO                 : {_ato_count:>6,} rows')

# ── 6D: SMURFING ──────────────────────────────────────────────────────────────
cfg = FRAUD_VOLUMES['smurfing']
for _ in range(cfg['num_groups']):
    acc       = random.choice(active_pool)
    cust      = acc_meta_fraud.loc[acc, 'customer_id']
    device    = account_device_map[acc]
    exit_bene = random.choice(high_risk_bene_pool)
    base_day  = random.randint(0, DATE_RANGE_DAYS - cfg['day_window'] - 1)
    for _ in range(random.randint(cfg['min_txns'], cfg['max_txns'])):
        ts  = random_ts(True, day_min=base_day, day_max=base_day + cfg['day_window'])
        amt = round(random.uniform(cfg['amount_min'], cfg['amount_max']), 2)
        ch  = random.choice(cfg['channels'])
        fraud_rows.append(build_row(txn_id, cust, acc, device, ts, amt, ch, 'smurfing', 1,
                                    beneficiary_id=exit_bene, force_cash=True))
        txn_id += 1

_sm_count = sum(1 for r in fraud_rows if r[_FRAUD_TYPE_IDX] == 'smurfing')
print(f'  Smurfing            : {_sm_count:>6,} rows')

# ── 6E: IDENTITY FRAUD ────────────────────────────────────────────────────────
cfg      = FRAUD_VOLUMES['identity_fraud']
new_accs = accounts.loc[
    (accounts['account_open_days'] < cfg['max_account_days']) &
    (~accounts['account_id'].isin(inactive_account_set)),
    'account_id'
].tolist()
sample_n = min(cfg['count'], len(new_accs))
for acc in random.sample(new_accs, sample_n):
    cust      = acc_meta_fraud.loc[acc, 'customer_id']
    device    = account_device_map[acc]
    exit_bene = random.choice(high_risk_bene_pool)
    ts        = random_ts(True, cfg['day_min'], cfg['day_max'])
    amt       = np.random.lognormal(cfg['amount_mean'], cfg['amount_sigma'])
    ch        = random.choice(cfg['channels'])
    fraud_rows.append(build_row(txn_id, cust, acc, device, ts, amt, ch, 'identity_fraud', 1,
                                beneficiary_id=exit_bene))
    txn_id += 1

_id_count = sum(1 for r in fraud_rows if r[_FRAUD_TYPE_IDX] == 'identity_fraud')
print(f'  Identity fraud      : {_id_count:>6,} rows')

# ── 6F: DORMANT-ACCOUNT FRAUD VARIANTS ───────────────────────────────────────
dormant_accs_list = list(dormant_account_set)

# 6F-i  Dormant ATO
cfg   = FRAUD_VOLUMES['dormant_ATO']
n_ato = max(1, int(len(dormant_accs_list) * cfg['frac_dormant_targeted']))
for acc in random.sample(dormant_accs_list, n_ato):
    cust       = acc_meta_fraud.loc[acc, 'customer_id']
    home_dev   = account_device_map[acc]
    other_devs = [d for d in all_devs if d != home_dev]
    new_dev    = random.choice(other_devs)
    exit_bene  = random.choice(high_risk_bene_pool)
    base_day   = random.randint(DORMANT_SECOND_WINDOW[0], DORMANT_SECOND_WINDOW[1] - 1)
    for _ in range(random.randint(*cfg['txns_per_account'])):
        ts  = random_ts(True, day_min=base_day,
                        day_max=min(base_day + 1, DORMANT_SECOND_WINDOW[1]))
        amt = np.random.lognormal(cfg['amount_mean'], cfg['amount_sigma'])
        ch  = random.choice(cfg['channels'])
        fraud_rows.append(build_row(txn_id, cust, acc, new_dev, ts, amt, ch,
                                    'dormant_ATO', 1, beneficiary_id=exit_bene))
        txn_id += 1

_dato_count = sum(1 for r in fraud_rows if r[_FRAUD_TYPE_IDX] == 'dormant_ATO')
print(f'  Dormant ATO         : {_dato_count:>6,} rows  ({n_ato} accounts)')

# 6F-ii  Dormant smurfing
cfg     = FRAUD_VOLUMES['dormant_smurfing']
n_smurf = max(1, int(len(dormant_accs_list) * cfg['frac_dormant_targeted']))
for acc in random.sample(dormant_accs_list, n_smurf):
    cust      = acc_meta_fraud.loc[acc, 'customer_id']
    device    = account_device_map[acc]
    exit_bene = random.choice(high_risk_bene_pool)
    base_day  = random.randint(DORMANT_SECOND_WINDOW[0],
                               DORMANT_SECOND_WINDOW[1] - cfg['day_window'] - 1)
    for _ in range(random.randint(cfg['min_txns'], cfg['max_txns'])):
        ts  = random_ts(True, day_min=base_day, day_max=base_day + cfg['day_window'])
        amt = round(random.uniform(cfg['amount_min'], cfg['amount_max']), 2)
        ch  = random.choice(cfg['channels'])
        fraud_rows.append(build_row(txn_id, cust, acc, device, ts, amt, ch,
                                    'dormant_smurfing', 1,
                                    beneficiary_id=exit_bene, force_cash=True))
        txn_id += 1

_dsm_count = sum(1 for r in fraud_rows if r[_FRAUD_TYPE_IDX] == 'dormant_smurfing')
print(f'  Dormant smurfing    : {_dsm_count:>6,} rows  ({n_smurf} accounts)')

# 6F-iii  Dormant-to-offshore
cfg        = FRAUD_VOLUMES['dormant_to_offshore']
n_offshore = max(1, int(len(dormant_accs_list) * cfg['frac_dormant_targeted']))
exit_pool  = offshore_crypto_pool if offshore_crypto_pool else high_risk_bene_pool
for acc in random.sample(dormant_accs_list, n_offshore):
    cust     = acc_meta_fraud.loc[acc, 'customer_id']
    device   = account_device_map[acc]
    base_day = random.randint(DORMANT_SECOND_WINDOW[0], DORMANT_SECOND_WINDOW[1] - 2)
    for _ in range(random.randint(*cfg['txns_per_account'])):
        exit_bene = random.choice(exit_pool)
        ts  = random_ts(True, day_min=base_day,
                        day_max=min(base_day + 2, DORMANT_SECOND_WINDOW[1]))
        amt = np.random.lognormal(cfg['amount_mean'], cfg['amount_sigma'])
        ch  = random.choice(cfg['channels'])
        fraud_rows.append(build_row(txn_id, cust, acc, device, ts, amt, ch,
                                    'dormant_to_offshore', 1,
                                    beneficiary_id=exit_bene))
        txn_id += 1

_doff_count = sum(1 for r in fraud_rows if r[_FRAUD_TYPE_IDX] == 'dormant_to_offshore')
print(f'  Dormant→offshore    : {_doff_count:>6,} rows  ({n_offshore} accounts)')

fraud_df_active = pd.DataFrame(fraud_rows, columns=TXN_COLS)

# ── Assertions ────────────────────────────────────────────────────────────────
assert fraud_df_active['fraud_type'].nunique() == 8, 'Expected exactly 8 active fraud typologies'
smurf_mask = fraud_df_active['fraud_type'].isin({'smurfing','dormant_smurfing'})
assert fraud_df_active.loc[smurf_mask, 'cash_flag'].eq(1).all(), 'Smurfing cash_flag must be 1'
assert not fraud_df_active['sender_account_id'].isin(inactive_account_set).any(), \
    '❌ Active-fraud pool must not include inactive accounts'

print(f'\n✅ Active/Dormant fraud complete : {len(fraud_df_active):,} rows across 8 typologies')
fraud_df_active['fraud_type'].value_counts()

In [ ]:
# ── A-7: Inactive-User Typologies (NEW) ──────────────────────────────────────
#
# Each inactive account has a precisely tracked «last_pre_silence_ts» which
# is the latest timestamp from its early cluster (Phase 1).
#
# All reactivation transactions begin on day ≥ INACTIVE_REACT_START, which
# is INACTIVE_EARLY_END + INACTIVE_GAP_DAYS, guaranteeing the 45-day gap.
#
# no_of_inactive_days (computed in Cell A-8) =
#     (reactivation_txn.timestamp.date − last_pre_silence_ts.date).days
# → always ≥ INACTIVE_GAP_DAYS (45) for reactivation rows
# → 0  for early-cluster rows (still in active phase)
# → NaN for all non-inactive accounts

# ── Shuffle & partition inactive pool into 4 mutually exclusive subsets ───────
inactive_list = list(inactive_account_set)
random.shuffle(inactive_list)
n_inact = len(inactive_list)

n_iato   = max(1, int(n_inact * INACTIVE_FRAUD_FRACS['inactive_ATO']))
n_ism    = max(1, int(n_inact * INACTIVE_FRAUD_FRACS['inactive_smurfing']))
n_ioff   = max(1, int(n_inact * INACTIVE_FRAUD_FRACS['inactive_to_offshore']))
n_ilegit = n_inact - n_iato - n_ism - n_ioff   # remainder → clean reactivation

subset_ato      = inactive_list[:n_iato]
subset_smurf    = inactive_list[n_iato : n_iato + n_ism]
subset_offshore = inactive_list[n_iato + n_ism : n_iato + n_ism + n_ioff]
subset_legit    = inactive_list[n_iato + n_ism + n_ioff :]

print(f'Inactive pool ({n_inact} accounts) — sub-pattern allocation:')
print(f'   inactive_ATO          : {len(subset_ato):>5}  ({len(subset_ato)/n_inact*100:.0f}%)')
print(f'   inactive_smurfing     : {len(subset_smurf):>5}  ({len(subset_smurf)/n_inact*100:.0f}%)')
print(f'   inactive_to_offshore  : {len(subset_offshore):>5}  ({len(subset_offshore)/n_inact*100:.0f}%)')
print(f'   inactive_legit (clean): {len(subset_legit):>5}  ({len(subset_legit)/n_inact*100:.0f}%)')
print()

# ── Record each account's last pre-silence timestamp ─────────────────────────
# We approximate it as the latest possible early-cluster day minus a small jitter.
inactive_last_ts_map: dict = {}
for acc in inactive_list:
    end_day = INACTIVE_EARLY_END - random.randint(0, 3)   # 0–3 days before window end
    inactive_last_ts_map[acc] = START_DATE + timedelta(
        days    = max(0, end_day),
        hours   = weighted_hour(fraud=False),
        minutes = random.randint(0, 59),
    )

inactive_rows: list = []

# ─────────────────────────────────────────────────────────────────────────────
# 7A: inactive_ATO
#     Account taken over by adversary after 45+ day silence.
#     Signals: foreign device, off-hours, large amounts, high-risk bene.
# ─────────────────────────────────────────────────────────────────────────────
cfg = FRAUD_VOLUMES['inactive_ATO']
for acc in subset_ato:
    cust      = acc_meta_fraud.loc[acc, 'customer_id']
    home_dev  = account_device_map[acc]
    other_dev = random.choice([d for d in all_devs if d != home_dev])
    exit_bene = random.choice(high_risk_bene_pool)
    for _ in range(random.randint(*cfg['txns_per_account'])):
        ts  = random_ts(fraud=True,
                        day_min=INACTIVE_REACT_START,
                        day_max=DATE_RANGE_DAYS)
        amt = np.random.lognormal(cfg['amount_mean'], cfg['amount_sigma'])
        ch  = random.choice(cfg['channels'])
        inactive_rows.append(build_row(
            txn_id, cust, acc, other_dev, ts, amt, ch, 'inactive_ATO', 1,
            beneficiary_id=exit_bene))
        txn_id += 1

_iato = sum(1 for r in inactive_rows if r[_FRAUD_TYPE_IDX] == 'inactive_ATO')
print(f'  inactive_ATO          : {_iato:>6,} rows')

# ─────────────────────────────────────────────────────────────────────────────
# 7B: inactive_smurfing
#     Burst of cash transactions just below ₹10k reporting threshold
#     within a 1–3 day window after the silence period ends.
# ─────────────────────────────────────────────────────────────────────────────
cfg = FRAUD_VOLUMES['inactive_smurfing']
for acc in subset_smurf:
    cust      = acc_meta_fraud.loc[acc, 'customer_id']
    device    = account_device_map[acc]     # own device (account re-used after takeover)
    exit_bene = random.choice(high_risk_bene_pool)
    base_day  = random.randint(
        INACTIVE_REACT_START,
        max(INACTIVE_REACT_START, DATE_RANGE_DAYS - cfg['day_window'] - 1)
    )
    for _ in range(random.randint(cfg['min_txns'], cfg['max_txns'])):
        ts  = random_ts(fraud=True, day_min=base_day,
                        day_max=min(base_day + cfg['day_window'], DATE_RANGE_DAYS))
        amt = round(random.uniform(cfg['amount_min'], cfg['amount_max']), 2)
        ch  = random.choice(cfg['channels'])
        inactive_rows.append(build_row(
            txn_id, cust, acc, device, ts, amt, ch, 'inactive_smurfing', 1,
            beneficiary_id=exit_bene, force_cash=True))
        txn_id += 1

_ism = sum(1 for r in inactive_rows if r[_FRAUD_TYPE_IDX] == 'inactive_smurfing')
print(f'  inactive_smurfing     : {_ism:>6,} rows')

# ─────────────────────────────────────────────────────────────────────────────
# 7C: inactive_to_offshore
#     Reactivated account immediately wires to crypto / offshore destination.
#     Signal: very large single transaction, high-risk beneficiary, web/mobile.
# ─────────────────────────────────────────────────────────────────────────────
cfg       = FRAUD_VOLUMES['inactive_to_offshore']
exit_pool = offshore_crypto_pool if offshore_crypto_pool else high_risk_bene_pool
for acc in subset_offshore:
    cust     = acc_meta_fraud.loc[acc, 'customer_id']
    device   = account_device_map[acc]
    base_day = random.randint(
        INACTIVE_REACT_START,
        max(INACTIVE_REACT_START, DATE_RANGE_DAYS - 3)
    )
    for _ in range(random.randint(*cfg['txns_per_account'])):
        exit_bene = random.choice(exit_pool)
        ts  = random_ts(fraud=True, day_min=base_day,
                        day_max=min(base_day + 2, DATE_RANGE_DAYS))
        amt = np.random.lognormal(cfg['amount_mean'], cfg['amount_sigma'])
        ch  = random.choice(cfg['channels'])
        inactive_rows.append(build_row(
            txn_id, cust, acc, device, ts, amt, ch, 'inactive_to_offshore', 1,
            beneficiary_id=exit_bene))
        txn_id += 1

_ioff = sum(1 for r in inactive_rows if r[_FRAUD_TYPE_IDX] == 'inactive_to_offshore')
print(f'  inactive_to_offshore  : {_ioff:>6,} rows')

# ─────────────────────────────────────────────────────────────────────────────
# 7D: inactive_legit  — clean reactivation (label=0)
#     Normal-sized amounts on the account's own home device.
#     Still carries no_of_inactive_days so AML analysts can study
#     benign reactivation patterns vs. fraudulent ones.
# ─────────────────────────────────────────────────────────────────────────────
for acc in subset_legit:
    kyc    = acc_meta_fraud.loc[acc, 'kyc_level']
    cust   = acc_meta_fraud.loc[acc, 'customer_id']
    device = account_device_map[acc]
    params = AMOUNT_BY_KYC.get(str(kyc).lower(), AMOUNT_DEFAULT)
    react_day = random.randint(INACTIVE_REACT_START, DATE_RANGE_DAYS)
    for _ in range(random.randint(2, 6)):
        ts  = random_ts(fraud=False, day_min=react_day,
                        day_max=min(react_day + 5, DATE_RANGE_DAYS))
        amt = np.random.lognormal(params['mean'], params['sigma'])
        ch  = random.choices(CHANNELS, weights=CHANNEL_PROBS)[0]
        if random.random() < INTERNAL_TRANSFER_PROB:
            inactive_rows.append(build_row(
                txn_id, cust, acc, device, ts, amt, ch, 'normal', 0,
                receiver_account_id=random.choice(all_account_ids)))
        else:
            inactive_rows.append(build_row(
                txn_id, cust, acc, device, ts, amt, ch, 'normal', 0,
                beneficiary_id=random.choice(account_beneficiaries_map[acc])))
        txn_id += 1

inactive_df = pd.DataFrame(inactive_rows, columns=TXN_COLS)
_ilegit = int((inactive_df['label'] == 0).sum())
print(f'  inactive_legit (clean): {_ilegit:>6,} rows')

# ── Assertions ────────────────────────────────────────────────────────────────
assert inactive_df['sender_account_id'].isin(inactive_account_set).all(), \
    '❌ Inactive rows must only come from inactive accounts'
ism_mask_df = inactive_df['fraud_type'] == 'inactive_smurfing'
if ism_mask_df.any():
    assert inactive_df.loc[ism_mask_df, 'cash_flag'].eq(1).all(), \
        '❌ inactive_smurfing cash_flag must be 1'

print(f'\n✅ Inactive typologies complete : {len(inactive_df):,} rows  '
      f'(fraud rate {inactive_df["label"].mean()*100:.1f}%)')
inactive_df['fraud_type'].value_counts()

In [ ]:
# ── A-8: Merge, Stamp no_of_inactive_days, Rebalance & FK-Enrichment ─────────

# 8a: Stack all three DataFrames ──────────────────────────────────────────────
combined = pd.concat([legit_df, fraud_df_active, inactive_df], ignore_index=True)
combined['timestamp'] = pd.to_datetime(combined['timestamp'])

# 8b: Stamp no_of_inactive_days ───────────────────────────────────────────────
#
# Column semantics:
#   NaN   — sender_account_id is NOT in inactive_account_set
#   0     — inactive account, but this row is in the PRE-SILENCE early cluster
#           (timestamp ≤ INACTIVE_EARLY_END)
#   ≥ 45  — reactivation row; value = exact calendar-day gap since last
#           pre-silence transaction (always ≥ INACTIVE_GAP_DAYS)

combined['no_of_inactive_days'] = np.nan

inact_mask = combined['sender_account_id'].isin(inactive_account_set)

def _compute_inactive_days(row):
    last_ts = inactive_last_ts_map.get(row['sender_account_id'])
    if last_ts is None:
        return np.nan
    delta = (row['timestamp'].date() - last_ts.date()).days
    if delta < INACTIVE_GAP_DAYS:
        # Row is in the early active phase — not a reactivation yet
        return 0
    return int(delta)

combined.loc[inact_mask, 'no_of_inactive_days'] = (
    combined.loc[inact_mask].apply(_compute_inactive_days, axis=1)
)

# Quick sanity check
react_check = combined[
    inact_mask & (combined['no_of_inactive_days'] >= INACTIVE_GAP_DAYS)
]
print(f'Inactive reactivation rows (no_of_inactive_days ≥ {INACTIVE_GAP_DAYS}): '
      f'{len(react_check):,}')
print(f'  gap range : {react_check["no_of_inactive_days"].min():.0f} – '
      f'{react_check["no_of_inactive_days"].max():.0f} days')
print(f'  mean gap  : {react_check["no_of_inactive_days"].mean():.1f} days')
print()

# 8c: Rebalance to TARGET_FRAUD_RATE ──────────────────────────────────────────
current_fraud = int(combined['label'].sum())
current_total = len(combined)
current_rate  = current_fraud / current_total
print(f'Pre-rebalance  : {current_rate:.4f} fraud rate ({current_fraud:,} / {current_total:,})')

# Oversample pool = ALL fraud rows (active + inactive)
all_fraud_df = pd.concat([
    fraud_df_active,
    inactive_df[inactive_df['label'] == 1]
], ignore_index=True)

if current_rate < TARGET_FRAUD_RATE:
    needed = int(TARGET_FRAUD_RATE * current_total / (1 - TARGET_FRAUD_RATE)) - current_fraud
    extra  = all_fraud_df.sample(n=needed, replace=True, random_state=42).copy()
    extra['transaction_id'] = [f'T{txn_id + i}_OS' for i in range(len(extra))]
    combined = pd.concat([combined, extra], ignore_index=True)
    combined['timestamp'] = pd.to_datetime(combined['timestamp'])
    print(f'Oversampled    : +{needed:,} fraud rows → target {TARGET_FRAUD_RATE:.0%}')

raw_txns = combined.sample(frac=1, random_state=42).reset_index(drop=True)
print(f'Post-rebalance : {raw_txns["label"].mean():.4f} fraud rate '
      f'({int(raw_txns["label"].sum()):,} / {len(raw_txns):,})')

# 8d: FK joins ────────────────────────────────────────────────────────────────
txns = raw_txns.copy()

acc_join_cols = [
    'account_id', 'avg_balance', 'account_open_days',
    'kyc_level', 'country_risk', 'income_bracket',
    'customer_risk_rating', 'pep_flag', 'occupation',
    'industry', 'account_type',
    'home_lat', 'home_lon', 'home_city',
    'shared_kyc_id', 'shared_phone_hash', 'shared_email_hash',
]
txns = txns.merge(
    accounts[acc_join_cols].rename(columns={'account_id': 'sender_account_id'}),
    on='sender_account_id', how='left'
)
dev_cols  = ['device_id', 'device_age_days', 'rooted_flag', 'os_type',
             'vpn_flag', 'emulator_flag']
txns = txns.merge(devices[dev_cols], on='device_id', how='left')

bene_cols = ['beneficiary_id', 'beneficiary_type', 'beneficiary_country_risk']
txns = txns.merge(beneficiaries[bene_cols], on='beneficiary_id', how='left')

print(f'\n✅ Merge + enrichment complete : {len(txns):,} rows × {txns.shape[1]} cols')
print(f'   account join      : +{len(acc_join_cols)-1} cols')
print(f'   device join       : +{len(dev_cols)-1} cols')
print(f'   bene join         : +{len(bene_cols)-1} cols')
print(f'   no_of_inactive_days  non-NaN : {txns["no_of_inactive_days"].notna().sum():,}')

In [ ]:
# ── A-9: IP Graph Layer ───────────────────────────────────────────────────────
# Enhancement for inactive typologies:
#   inactive_ATO rows → always ip_type='risk' (adversary TOR/VPN usage)
#   inactive reactivation rows (fraud) → +0.05 ip_risk_score bump

print('Computing IP Graph Layer...')

ROAM_CITIES = [
    (28.6139,77.2090),(19.0760,72.8777),(12.9716,77.5946),(22.5726,88.3639),
    (17.3850,78.4867),(13.0827,80.2707),(18.5204,73.8567),(23.0225,72.5714),
    (26.9124,75.7873),(21.1702,72.8311),
]

rng_ip   = np.random.default_rng(RANDOM_SEED + 99)
RISK_IPS = [
    f'{pfx}.{rng_ip.integers(1,254)}.{rng_ip.integers(1,254)}'
    for pfx in HIGH_RISK_IP_PREFIXES for _ in range(20)
]

def _home_ip(customer_id, account_id):
    h = hash(str(customer_id) + str(account_id)) & 0xFFFF
    return f'10.{(h >> 8) & 0xFF}.{h & 0xFF}.{hash(str(account_id)) & 0xFF}'

rng_geo = np.random.default_rng(RANDOM_SEED + 77)
ip_addresses, ip_risk_scores, geo_lats, geo_lons = [], [], [], []

for _, row in txns.iterrows():
    cid    = row['customer_id']
    aid    = row['sender_account_id']
    vpn    = int(row.get('vpn_flag',      0) or 0)
    rooted = int(row.get('rooted_flag',   0) or 0)
    emu    = int(row.get('emulator_flag', 0) or 0)
    night  = int(row.get('is_night',      0) or 0)
    crisk  = row.get('country_risk', 'low')  or 'low'
    kyc    = row.get('kyc_level',  'medium') or 'medium'
    ftype  = row.get('fraud_type', '')       or ''
    inact_days = row.get('no_of_inactive_days', np.nan)

    # inactive_ATO always routed through adversary-controlled risk IPs
    if vpn == 1 or ftype == 'inactive_ATO':
        ip_type = 'risk'
    else:
        r = float(rng_geo.random())
        ip_type = ('home' if r < IP_HOME_FRAC
                   else 'roam' if r < IP_HOME_FRAC + IP_ROAM_FRAC
                   else 'risk')

    if ip_type == 'home':
        ip = _home_ip(cid, aid)
    elif ip_type == 'roam':
        ip = (f'{int(rng_geo.integers(100,223))}.'
              f'{int(rng_geo.integers(0,255))}.'
              f'{int(rng_geo.integers(0,255))}.'
              f'{int(rng_geo.integers(1,254))}')
    else:
        ip = RISK_IPS[int(rng_geo.integers(0, len(RISK_IPS)))]
    ip_addresses.append(ip)

    base  = (IP_HOME_BASE_SCORE if ip_type == 'home'
             else IP_ROAM_BASE_SCORE if ip_type == 'roam'
             else IP_RISK_BASE_SCORE)
    score = base
    score += 0.15 if vpn    else 0
    score += 0.10 if rooted else 0
    score += 0.10 if emu    else 0
    score += 0.10 if night  else 0
    score += 0.05 if str(crisk).lower() == 'high' else 0
    score -= 0.05 if str(kyc).lower()   == 'high' else 0
    # Extra bump: inactive fraud reactivation (not clean legit)
    if (pd.notna(inact_days) and
            inact_days >= INACTIVE_GAP_DAYS and
            ftype not in ('normal', '')):
        score += 0.05
    ip_risk_scores.append(round(min(max(score, 0.0), 1.0), 4))

    if ip_type == 'home':
        if cust_geo_map is not None and cid in cust_geo_map.index:
            hlat = float(cust_geo_map.loc[cid, 'home_lat'])
            hlon = float(cust_geo_map.loc[cid, 'home_lon'])
        else:
            hlat, hlon = 20.5937, 78.9629
        geo_lats.append(round(hlat + float(rng_geo.normal(0, 0.05)), 6))
        geo_lons.append(round(hlon + float(rng_geo.normal(0, 0.05)), 6))
    elif ip_type == 'roam':
        city = ROAM_CITIES[int(rng_geo.integers(0, len(ROAM_CITIES)))]
        geo_lats.append(round(city[0] + float(rng_geo.normal(0, 0.3)), 6))
        geo_lons.append(round(city[1] + float(rng_geo.normal(0, 0.3)), 6))
    else:
        geo_lats.append(round(float(rng_geo.uniform(-60, 70)), 6))
        geo_lons.append(round(float(rng_geo.uniform(-180, 180)), 6))

txns['ip_address']    = ip_addresses
txns['ip_risk_score'] = ip_risk_scores
txns['geo_lat']       = geo_lats
txns['geo_lon']       = geo_lons

print(f'✅ IP Graph Layer added: {txns.shape[1]} cols')
print(f'   home IPs   (score ≤ 0.20): {(txns["ip_risk_score"] <= 0.20).sum():,}')
print(f'   roam IPs   (0.20–0.69)   : {((txns["ip_risk_score"] > 0.20) & (txns["ip_risk_score"] < 0.70)).sum():,}')
print(f'   risk IPs   (score ≥ 0.70): {(txns["ip_risk_score"] >= 0.70).sum():,}')

In [ ]:
# ── A-10: Shared Identity Enrichment ─────────────────────────────────────────
# Original typologies: unchanged.
# Inactive typologies: distinct prefix tokens to avoid ID collision.

def _group_id(prefix, key, n_groups=200):
    h = int(hashlib.md5(str(key).encode()).hexdigest(), 16)
    return f'{prefix}_{h % n_groups:05d}'

fraud_mask = txns['label'] == 1
ft         = txns['fraud_type']

# ── Original 8 typologies (logic UNCHANGED) ───────────────────────────────────
mr_mask = fraud_mask & (ft == 'mule_ring')
txns.loc[mr_mask, 'shared_kyc_id'] = (
    txns.loc[mr_mask, 'synthetic_flow_id'].apply(lambda fid: _group_id('KYC', fid))
)

ly_mask = fraud_mask & (ft == 'layering')
txns.loc[ly_mask, 'shared_phone_hash'] = (
    txns.loc[ly_mask, 'synthetic_flow_id'].apply(lambda fid: _group_id('PHONE', fid))
)

sm_mask = fraud_mask & (ft == 'smurfing')
txns.loc[sm_mask, 'shared_email_hash'] = (
    txns.loc[sm_mask, 'sender_account_id'].apply(lambda aid: _group_id('EMAIL', aid))
)

id_mask = fraud_mask & (ft == 'identity_fraud')
txns.loc[id_mask, 'shared_email_hash'] = (
    txns.loc[id_mask, 'customer_id'].apply(lambda cid: _group_id('EMAIL', cid, n_groups=50))
)
txns.loc[id_mask, 'shared_phone_hash'] = (
    txns.loc[id_mask, 'customer_id'].apply(lambda cid: _group_id('PHONE', cid, n_groups=50))
)

ato_mask = fraud_mask & ft.isin({'ATO', 'dormant_ATO'})
txns.loc[ato_mask, 'shared_email_hash'] = (
    txns.loc[ato_mask, 'sender_account_id'].apply(lambda aid: _group_id('EMAIL', aid, n_groups=300))
)

dsm_mask = fraud_mask & (ft == 'dormant_smurfing')
txns.loc[dsm_mask, 'shared_kyc_id'] = (
    txns.loc[dsm_mask, 'sender_account_id'].apply(lambda aid: _group_id('KYC', aid, n_groups=100))
)
txns.loc[dsm_mask, 'shared_phone_hash'] = (
    txns.loc[dsm_mask, 'sender_account_id'].apply(lambda aid: _group_id('PHONE', aid, n_groups=100))
)

doff_mask = fraud_mask & (ft == 'dormant_to_offshore')
txns.loc[doff_mask, 'shared_kyc_id'] = (
    txns.loc[doff_mask, 'sender_account_id'].apply(lambda aid: _group_id('KYC', aid, n_groups=50))
)

# ── NEW: Inactive typology shared-identity tokens ─────────────────────────────
# Prefixes are distinct (INACT_*) to avoid confusion with dormant / active pools.

# inactive_ATO → shared email (same adversarial takeover infrastructure)
iato_mask = fraud_mask & (ft == 'inactive_ATO')
txns.loc[iato_mask, 'shared_email_hash'] = (
    txns.loc[iato_mask, 'sender_account_id']
    .apply(lambda aid: _group_id('INACT_EMAIL_ATO', aid, n_groups=400))
)

# inactive_smurfing → shared phone + email (same cash-structuring operative)
ism_mask = fraud_mask & (ft == 'inactive_smurfing')
txns.loc[ism_mask, 'shared_phone_hash'] = (
    txns.loc[ism_mask, 'sender_account_id']
    .apply(lambda aid: _group_id('INACT_PHONE_SM', aid, n_groups=150))
)
txns.loc[ism_mask, 'shared_email_hash'] = (
    txns.loc[ism_mask, 'sender_account_id']
    .apply(lambda aid: _group_id('INACT_EMAIL_SM', aid, n_groups=150))
)

# inactive_to_offshore → shared KYC + phone (same shell-entity documents)
ioff_mask = fraud_mask & (ft == 'inactive_to_offshore')
txns.loc[ioff_mask, 'shared_kyc_id'] = (
    txns.loc[ioff_mask, 'sender_account_id']
    .apply(lambda aid: _group_id('INACT_KYC_OFF', aid, n_groups=75))
)
txns.loc[ioff_mask, 'shared_phone_hash'] = (
    txns.loc[ioff_mask, 'sender_account_id']
    .apply(lambda aid: _group_id('INACT_PHONE_OFF', aid, n_groups=75))
)

print('✅ Shared Identity Enrichment complete')
print(f'   shared_kyc_id      non-null : {txns["shared_kyc_id"].notna().sum():,}')
print(f'   shared_phone_hash  non-null : {txns["shared_phone_hash"].notna().sum():,}')
print(f'   shared_email_hash  non-null : {txns["shared_email_hash"].notna().sum():,}')
print('\nInactive sub-pattern coverage:')
print(f'   inactive_ATO   → email  : {txns.loc[iato_mask, "shared_email_hash"].notna().sum():,}')
print(f'   inactive_smurf → phone  : {txns.loc[ism_mask,  "shared_phone_hash"].notna().sum():,}')
print(f'   inactive_offsh → kyc    : {txns.loc[ioff_mask, "shared_kyc_id"].notna().sum():,}')

In [ ]:
# ── A-11: Validation Dashboard ────────────────────────────────────────────────

SEP = '=' * 68
sep = '─' * 68

print(SEP)
print('DATASET VALIDATION REPORT')
print(SEP)

total_rows   = len(txns)
fraud_n      = int(txns['label'].sum())
legit_n      = total_rows - fraud_n

print(f'  Total rows             : {total_rows:>10,}')
print(f'  Fraud rows (label=1)   : {fraud_n:>10,}  ({fraud_n/total_rows*100:.2f}%)')
print(f'  Legit rows (label=0)   : {legit_n:>10,}  ({legit_n/total_rows*100:.2f}%)')
print(f'  Total columns          : {txns.shape[1]:>10}')
print()

print(sep)
print('TYPOLOGY BREAKDOWN')
print(sep)
tvc = txns['fraud_type'].value_counts(dropna=False)
for typ, cnt in tvc.items():
    pct  = cnt / total_rows * 100
    pool = ('  ← INACTIVE' if str(typ).startswith('inactive_')
            else '  ← DORMANT' if str(typ).startswith('dormant_')
            else '')
    print(f'  {str(typ):<30}  {cnt:>8,}  ({pct:5.2f}%){pool}')
print()

print(sep)
print('INACTIVE ACCOUNT STATISTICS')
print(sep)
inact_txns = txns[txns['sender_account_id'].isin(inactive_account_set)]
react_txns = inact_txns[inact_txns['no_of_inactive_days'] >= INACTIVE_GAP_DAYS]
early_txns = inact_txns[inact_txns['no_of_inactive_days'] == 0]

print(f'  Inactive accounts sampled   : {len(inactive_account_set):,} '
      f'({len(inactive_account_set)/len(all_account_ids)*100:.1f}% of all accounts)')
print(f'  Total rows from inactive    : {len(inact_txns):,}')
print(f'    Pre-silence early rows    : {len(early_txns):,}  (no_of_inactive_days = 0)')
print(f'    Reactivation rows (≥45d)  : {len(react_txns):,}')
print(f'      Fraud reactivation      : {int(react_txns["label"].sum()):,}')
print(f'      Legit reactivation      : {int((react_txns["label"]==0).sum()):,}')
if len(react_txns):
    print(f'  no_of_inactive_days stats   :')
    print(f'    min    = {react_txns["no_of_inactive_days"].min():.0f} days')
    print(f'    median = {react_txns["no_of_inactive_days"].median():.0f} days')
    print(f'    mean   = {react_txns["no_of_inactive_days"].mean():.1f} days')
    print(f'    max    = {react_txns["no_of_inactive_days"].max():.0f} days')
print()

print(sep)
print('OVERLAP & INTEGRITY CHECKS')
print(sep)
overlap = dormant_account_set & inactive_account_set
print(f'  dormant ∩ inactive          : {len(overlap)}  '
      f'→ {"✅ CLEAN" if not overlap else "❌ OVERLAP"}')
cross = fraud_df_active['sender_account_id'].isin(inactive_account_set).sum()
print(f'  Active-fraud rows in inact  : {cross}  '
      f'→ {"✅ CLEAN" if cross == 0 else "❌ OVERLAP"}')
inact_in_active = inactive_df['sender_account_id'].isin(dormant_account_set).sum()
print(f'  Inactive rows in dormant    : {inact_in_active}  '
      f'→ {"✅ CLEAN" if inact_in_active == 0 else "❌ OVERLAP"}')
print()

print(sep)
print('no_of_inactive_days COLUMN SUMMARY')
print(sep)
print(f'  dtype   : {txns["no_of_inactive_days"].dtype}')
print(f'  NaN     : {txns["no_of_inactive_days"].isna().sum():,}  '
      f'(non-inactive accounts)')
print(f'  = 0     : {(txns["no_of_inactive_days"]==0).sum():,}  '
      f'(inactive acct early-cluster rows)')
print(f'  ≥ 45    : {(txns["no_of_inactive_days"]>=INACTIVE_GAP_DAYS).sum():,}  '
      f'(reactivation rows)')
print(SEP)

# Sample of reactivation rows
txns[['sender_account_id','timestamp','fraud_type','label','no_of_inactive_days']]\
    .query('no_of_inactive_days >= 45')\
    .sort_values('no_of_inactive_days', ascending=False)\
    .head(12)

In [ ]:
# ── A-12: Save Stage-1 Output ─────────────────────────────────────────────────

txns.to_parquet(OUTPUT_DIR / 'txns_stage1.parquet', index=False)

# Dormant set (for Notebook 2b — original contract)
pd.DataFrame({'account_id': sorted(dormant_account_set)}).to_csv(
    OUTPUT_DIR / 'dormant_account_set.csv', index=False
)

# Inactive set (NEW — for Notebook 2b rolling-feature context)
pd.DataFrame({
    'account_id'    : sorted(inactive_account_set),
    'last_active_ts': [
        inactive_last_ts_map[a].isoformat()
        for a in sorted(inactive_account_set)
    ],
    'inactive_gap_days': INACTIVE_GAP_DAYS,
    'react_start_day'  : INACTIVE_REACT_START,
}).to_csv(OUTPUT_DIR / 'inactive_account_set.csv', index=False)

size_mb = (OUTPUT_DIR / 'txns_stage1.parquet').stat().st_size / 1024**2

print('✅ All outputs saved')
print(f'   txns_stage1.parquet        → {len(txns):,} rows × {txns.shape[1]} cols | {size_mb:.1f} MB')
print(f'   dormant_account_set.csv    → {len(dormant_account_set):,} accounts')
print(f'   inactive_account_set.csv   → {len(inactive_account_set):,} accounts')
print()
print('Final column list:')
for c in txns.columns:
    tag = '  ← NEW' if c == 'no_of_inactive_days' else ''
    print(f'   {c}{tag}')
print()
print('Next step → Run  02b_rolling_features.ipynb')